In [10]:
"""
Bank Account Management System
==============================
A comprehensive banking system that allows users to:
- Create and manage bank accounts
- Perform transactions (deposit, withdraw, transfer)
- View transaction history and statistics
- Persist data using file handling

Author: [Your Name]
Date: [Current Date]
"""

# ============================================================================
# IMPORTS
# ============================================================================
import pickle          # For saving/loading data to files
import os              # For file existence checking
import numpy as np     # For statistical calculations
from datetime import datetime  # For transaction timestamps
import random          # For potential future features

# ============================================================================
# BANKACCOUNT CLASS
# ============================================================================
class BankAccount:
    """
    Represents a single bank account with transaction capabilities.
    
    Attributes:
        holder_name (str): Name of the account holder
        account_number (int): Unique account identifier (auto-generated)
        account_type (str): Type of account ('savings' or 'current')
        balance (float): Current account balance
        transaction_history (list): List of all transactions
        created_date (datetime): Date when account was created
    
    Class Attributes:
        account_counter (int): Tracks the next account number to assign
    """
    
    # Class variable - shared across all instances
    # Starts at 1000 and increments for each new account
    account_counter = 1000
    
    def __init__(self, holder_name, account_type, initial_balance=0):
        """
        Initialize a new bank account.
        
        Args:
            holder_name (str): Name of the account holder
            account_type (str): Type of account ('savings' or 'current')
            initial_balance (float): Initial deposit amount (default: 0)
        
        The constructor automatically:
        - Assigns a unique account number
        - Initializes transaction history
        - Records the initial deposit if any
        """
        # Instance attributes - unique to each account
        self.holder_name = holder_name
        self.account_number = BankAccount.account_counter
        
        # Increment counter for next account
        BankAccount.account_counter += 1
        
        # Store account type in lowercase for consistency
        self.account_type = account_type.lower()
        self.balance = initial_balance
        
        # List to store all transactions as dictionaries
        self.transaction_history = []
        
        # Record when account was created
        self.created_date = datetime.now()
        
        # Record initial deposit if any amount was provided
        if initial_balance > 0:
            self.transaction_history.append({
                'date': datetime.now(),
                'type': 'Initial Deposit',
                'amount': initial_balance,
                'balance_after': self.balance
            })
    
    def get_account_details(self):
        """
        Return formatted account details.
        
        Returns:
            dict: Dictionary containing account information
        
        This method provides a clean view of account information
        without exposing internal attributes directly.
        """
        return {
            'Account Holder': self.holder_name,
            'Account Number': self.account_number,
            'Account Type': self.account_type.capitalize(),
            'Current Balance': f"${self.balance:.2f}",  # Format with 2 decimal places
            'Created Date': self.created_date.strftime('%Y-%m-%d %H:%M:%S')
        }
    
    def deposit(self, amount):
        """
        Deposit money into the account.
        
        Args:
            amount (float): Amount to deposit
            
        Returns:
            bool: True if successful, False otherwise
        
        Validation:
        - Amount must be positive
        
        Process:
        1. Validate the amount
        2. Add amount to balance
        3. Record transaction in history
        4. Display confirmation message
        """
        try:
            # VALIDATION: Check if amount is positive
            if amount <= 0:
                print("Error: Deposit amount must be positive!")
                return False
            
            # UPDATE BALANCE
            self.balance += amount
            
            # RECORD TRANSACTION with all relevant details
            self.transaction_history.append({
                'date': datetime.now(),           # When transaction occurred
                'type': 'Deposit',                # Type of transaction
                'amount': amount,                 # How much was deposited
                'balance_after': self.balance     # Balance after transaction
            })
            
            # CONFIRMATION MESSAGE
            print(f"✓ Successfully deposited ${amount:.2f}")
            print(f"New balance: ${self.balance:.2f}")
            return True
            
        except Exception as e:
            # Catch any unexpected errors
            print(f"Error during deposit: {e}")
            return False

    def withdraw(self, amount):
        """
        Withdraw money from the account.
        
        Args:
            amount (float): Amount to withdraw
            
        Returns:
            bool: True if successful, False otherwise
        
        Validation:
        - Amount must be positive
        - Account must have sufficient funds
        
        Process:
        1. Validate the amount
        2. Check sufficient balance
        3. Deduct amount from balance
        4. Record transaction in history
        5. Display confirmation message
        """
        try:
            # VALIDATION 1: Check if amount is positive
            if amount <= 0:
                print("Error: Withdrawal amount must be positive!")
                return False
            
            # VALIDATION 2: Check if sufficient funds available
            if amount > self.balance:
                print(f"Error: Insufficient funds! Current balance: ${self.balance:.2f}")
                return False
            
            # UPDATE BALANCE (subtract withdrawal amount)
            self.balance -= amount
            
            # RECORD TRANSACTION
            self.transaction_history.append({
                'date': datetime.now(),
                'type': 'Withdrawal',
                'amount': amount,
                'balance_after': self.balance
            })
            
            # CONFIRMATION MESSAGE
            print(f"✓ Successfully withdrew ${amount:.2f}")
            print(f"New balance: ${self.balance:.2f}")
            return True
            
        except Exception as e:
            print(f"Error during withdrawal: {e}")
            return False

    def transfer(self, recipient_account, amount):
        """
        Transfer money to another account.
        
        Args:
            recipient_account (BankAccount): The account to transfer to
            amount (float): Amount to transfer
            
        Returns:
            bool: True if successful, False otherwise
        
        Validation:
        - Amount must be positive
        - Recipient account must be valid
        - Cannot transfer to same account
        - Sender must have sufficient funds
        
        Process:
        1. Validate all conditions
        2. Deduct from sender's balance
        3. Add to recipient's balance
        4. Record transaction in BOTH accounts
        5. Display confirmation message
        """
        try:
            # VALIDATION 1: Check if amount is positive
            if amount <= 0:
                print("Error: Transfer amount must be positive!")
                return False
            
            # VALIDATION 2: Check if recipient account is valid BankAccount object
            if not isinstance(recipient_account, BankAccount):
                print("Error: Invalid recipient account!")
                return False
            
            # VALIDATION 3: Prevent transferring to same account
            if self.account_number == recipient_account.account_number:
                print("Error: Cannot transfer to the same account!")
                return False
            
            # VALIDATION 4: Check if sufficient funds available
            if amount > self.balance:
                print(f"Error: Insufficient funds! Current balance: ${self.balance:.2f}")
                return False
            
            # PERFORM TRANSFER (atomic operation - both accounts updated)
            self.balance -= amount                    # Deduct from sender
            recipient_account.balance += amount       # Add to recipient
            
            # RECORD TRANSACTION IN SENDER'S ACCOUNT
            self.transaction_history.append({
                'date': datetime.now(),
                'type': 'Transfer Out',
                'amount': amount,
                'to_account': recipient_account.account_number,  # Track recipient
                'balance_after': self.balance
            })
            
            # RECORD TRANSACTION IN RECIPIENT'S ACCOUNT
            recipient_account.transaction_history.append({
                'date': datetime.now(),
                'type': 'Transfer In',
                'amount': amount,
                'from_account': self.account_number,  # Track sender
                'balance_after': recipient_account.balance
            })
            
            # CONFIRMATION MESSAGE
            print(f"✓ Successfully transferred ${amount:.2f} to Account {recipient_account.account_number}")
            print(f"Your new balance: ${self.balance:.2f}")
            return True
            
        except Exception as e:
            print(f"Error during transfer: {e}")
            return False
    
    def view_transaction_history(self):
        """
        Display complete transaction history in a formatted table.
        
        This method provides a comprehensive view of all transactions
        including date, type, amount, resulting balance, and additional
        details for transfers (from/to account numbers).
        """
        # Check if there are any transactions
        if not self.transaction_history:
            print("No transactions found for this account.")
            return
        
        # HEADER
        print("\n" + "=" * 100)
        print(f"Transaction History for Account {self.account_number} - {self.holder_name}")
        print("=" * 100)
        
        # COLUMN HEADERS
        print(f"{'Date':<20} {'Type':<20} {'Amount':<15} {'Balance After':<15} {'Details':<20}")
        print("-" * 100)
        
        # ITERATE THROUGH ALL TRANSACTIONS
        for trans in self.transaction_history:
            # Format date for readability
            date_str = trans['date'].strftime('%Y-%m-%d %H:%M:%S')
            trans_type = trans['type']
            amount = f"${trans['amount']:.2f}"
            balance = f"${trans['balance_after']:.2f}"
            
            # Add additional details for transfers
            details = ""
            if 'to_account' in trans:
                details = f"To: {trans['to_account']}"
            elif 'from_account' in trans:
                details = f"From: {trans['from_account']}"
            
            # Print formatted row
            print(f"{date_str:<20} {trans_type:<20} {amount:<15} {balance:<15} {details:<20}")
        
        # FOOTER
        print("=" * 100)
        print(f"Total Transactions: {len(self.transaction_history)}\n")

    def get_transaction_statistics(self):
        """
        Generate summary statistics using NumPy.
        
        Returns:
            dict: Dictionary containing various statistics or None if no transactions
        
        Statistics calculated:
        - Total and count of deposits, withdrawals, transfers
        - Average, largest, smallest transaction amounts
        - Standard deviation
        - Current balance and net change
        
        Uses NumPy for efficient numerical calculations.
        """
        # Check if there are transactions to analyze
        if not self.transaction_history:
            print("No transactions to analyze.")
            return None
        
        # INITIALIZE LISTS for different transaction types
        deposits = []
        withdrawals = []
        transfers_out = []
        transfers_in = []
        all_amounts = []
        
        # CATEGORIZE TRANSACTIONS by type
        for trans in self.transaction_history:
            amount = trans['amount']
            trans_type = trans['type']
            
            # All amounts (for overall statistics)
            all_amounts.append(amount)
            
            # Categorize by transaction type
            if trans_type in ['Deposit', 'Initial Deposit']:
                deposits.append(amount)
            elif trans_type == 'Withdrawal':
                withdrawals.append(amount)
            elif trans_type == 'Transfer Out':
                transfers_out.append(amount)
            elif trans_type == 'Transfer In':
                transfers_in.append(amount)
        
        # CONVERT TO NUMPY ARRAYS for efficient calculations
        # Use [0] as default if list is empty to avoid errors
        all_amounts_np = np.array(all_amounts)
        deposits_np = np.array(deposits) if deposits else np.array([0])
        withdrawals_np = np.array(withdrawals) if withdrawals else np.array([0])
        transfers_out_np = np.array(transfers_out) if transfers_out else np.array([0])
        transfers_in_np = np.array(transfers_in) if transfers_in else np.array([0])
        
        # CALCULATE STATISTICS using NumPy functions
        stats = {
            # Counts
            'total_transactions': len(self.transaction_history),
            'count_deposits': len(deposits),
            'count_withdrawals': len(withdrawals),
            'count_transfers_out': len(transfers_out),
            'count_transfers_in': len(transfers_in),
            
            # Totals (using np.sum for summation)
            'total_deposits': np.sum(deposits_np),
            'total_withdrawals': np.sum(withdrawals_np),
            'total_transfers_out': np.sum(transfers_out_np),
            'total_transfers_in': np.sum(transfers_in_np),
            
            # Statistical measures
            'average_transaction': np.mean(all_amounts_np),      # Mean
            'largest_transaction': np.max(all_amounts_np),       # Maximum
            'smallest_transaction': np.min(all_amounts_np),      # Minimum
            'std_deviation': np.std(all_amounts_np),             # Standard deviation
            
            # Current state
            'current_balance': self.balance
        }
        
        return stats

    def display_statistics(self):
        """
        Display formatted statistics report.
        
        This method calls get_transaction_statistics() and presents
        the data in a user-friendly format with clear sections.
        """
        # Get statistics dictionary
        stats = self.get_transaction_statistics()
        
        # Exit if no statistics available
        if stats is None:
            return
        
        # HEADER
        print("\n" + "=" * 70)
        print(f"Account Statistics for {self.holder_name} (Account #{self.account_number})")
        print("=" * 70)
        
        # SECTION 1: Transaction Summary
        print(f"\n{'TRANSACTION SUMMARY':<40}")
        print("-" * 70)
        print(f"  Total Transactions: {stats['total_transactions']}")
        print(f"  Deposits: {stats['count_deposits']} (Total: ${stats['total_deposits']:.2f})")
        print(f"  Withdrawals: {stats['count_withdrawals']} (Total: ${stats['total_withdrawals']:.2f})")
        print(f"  Transfers Out: {stats['count_transfers_out']} (Total: ${stats['total_transfers_out']:.2f})")
        print(f"  Transfers In: {stats['count_transfers_in']} (Total: ${stats['total_transfers_in']:.2f})")
        
        # SECTION 2: Financial Statistics
        print(f"\n{'FINANCIAL STATISTICS':<40}")
        print("-" * 70)
        print(f"  Current Balance: ${stats['current_balance']:.2f}")
        print(f"  Average Transaction: ${stats['average_transaction']:.2f}")
        print(f"  Largest Transaction: ${stats['largest_transaction']:.2f}")
        print(f"  Smallest Transaction: ${stats['smallest_transaction']:.2f}")
        print(f"  Standard Deviation: ${stats['std_deviation']:.2f}")
        
        # CALCULATE NET CHANGE (money in - money out)
        net_change = (stats['total_deposits'] + stats['total_transfers_in'] - 
                      stats['total_withdrawals'] - stats['total_transfers_out'])
        print(f"  Net Change: ${net_change:.2f}")
        
        # FOOTER
        print("=" * 70 + "\n")
    
    def __str__(self):
        """
        String representation of the account.
        
        This special method is called when you use print() or str()
        on a BankAccount object.
        
        Returns:
            str: Formatted string with key account information
        """
        return f"Account {self.account_number} - {self.holder_name} ({self.account_type}): ${self.balance:.2f}"


# Confirmation message
print("✓ BankAccount class loaded successfully!")
print("  - Supports: Deposits, Withdrawals, Transfers")
print("  - Features: Transaction history, Statistics with NumPy")

✓ BankAccount class loaded successfully!
  - Supports: Deposits, Withdrawals, Transfers
  - Features: Transaction history, Statistics with NumPy


In [11]:
# ============================================================================
# BANKSYSTEM CLASS
# ============================================================================
class BankSystem:
    """
    Manages multiple bank accounts and handles file operations.
    
    This class serves as the main controller for the banking system,
    providing functionality to:
    - Create and manage multiple accounts
    - Save/load data using pickle (file persistence)
    - Generate system-wide reports
    - Search and retrieve accounts
    
    Attributes:
        filename (str): Name of the file to store account data
        accounts (dict): Dictionary mapping account_number -> BankAccount
    """
    
    def __init__(self, filename='bank_accounts.pkl'):
        """
        Initialize the bank system and load existing accounts.
        
        Args:
            filename (str): Name of the pickle file (default: 'bank_accounts.pkl')
        
        The constructor automatically attempts to load existing
        account data from the file if it exists.
        """
        self.filename = filename
        
        # Dictionary to store accounts: {account_number: BankAccount_object}
        # Using dictionary for O(1) lookup time
        self.accounts = {}
        
        # Load existing accounts from file
        self.load_accounts()
    
    def load_accounts(self):
        """
        Load accounts from pickle file if it exists.
        
        File Structure:
        The pickle file contains a dictionary with:
        - 'accounts': Dictionary of all BankAccount objects
        - 'account_counter': Last used account number
        
        This method:
        1. Checks if file exists
        2. Loads data using pickle.load()
        3. Restores accounts dictionary
        4. Restores account_counter to maintain unique account numbers
        """
        try:
            # CHECK if file exists
            if os.path.exists(self.filename):
                # OPEN file in binary read mode ('rb')
                with open(self.filename, 'rb') as file:
                    # LOAD data using pickle
                    data = pickle.load(file)
                    
                    # RESTORE accounts dictionary
                    self.accounts = data['accounts']
                    
                    # RESTORE account counter to continue numbering
                    # This ensures new accounts don't get duplicate numbers
                    BankAccount.account_counter = data['account_counter']
                
                print(f"✓ Loaded {len(self.accounts)} accounts from file.")
            else:
                # File doesn't exist - this is a fresh start
                print("No existing account file found. Starting fresh.")
                self.accounts = {}
                
        except Exception as e:
            # Handle any errors during loading
            print(f"Error loading accounts: {e}")
            self.accounts = {}
    
    def save_accounts(self):
        """
        Save all accounts to pickle file.
        
        File Structure:
        Saves a dictionary containing:
        - 'accounts': All BankAccount objects
        - 'account_counter': Current counter value
        
        Returns:
            bool: True if successful, False otherwise
        
        Using pickle allows us to save Python objects directly
        without converting to JSON or other formats.
        """
        try:
            # PREPARE data structure for saving
            data = {
                'accounts': self.accounts,
                'account_counter': BankAccount.account_counter
            }
            
            # OPEN file in binary write mode ('wb')
            with open(self.filename, 'wb') as file:
                # SAVE data using pickle
                pickle.dump(data, file)
            
            print(f"✓ Saved {len(self.accounts)} accounts to file.")
            return True
            
        except Exception as e:
            print(f"Error saving accounts: {e}")
            return False
    
    def create_account(self, holder_name, account_type, initial_balance=0):
        """
        Create a new bank account and add it to the system.
        
        Args:
            holder_name (str): Name of account holder
            account_type (str): 'savings' or 'current'
            initial_balance (float): Initial deposit (default: 0)
            
        Returns:
            BankAccount: The newly created account or None if failed
        
        Validation:
        - Name cannot be empty
        - Account type must be 'savings' or 'current'
        - Initial balance cannot be negative
        
        Process:
        1. Validate all inputs
        2. Create new BankAccount object
        3. Add to accounts dictionary
        4. Auto-save to file
        5. Display confirmation
        """
        try:
            # VALIDATION 1: Check if name is provided
            if not holder_name or not holder_name.strip():
                print("Error: Account holder name cannot be empty!")
                return None
            
            # VALIDATION 2: Check account type
            if account_type.lower() not in ['savings', 'current']:
                print("Error: Account type must be 'savings' or 'current'!")
                return None
            
            # VALIDATION 3: Check initial balance
            if initial_balance < 0:
                print("Error: Initial balance cannot be negative!")
                return None
            
            # CREATE new BankAccount object
            new_account = BankAccount(holder_name, account_type, initial_balance)
            
            # ADD to accounts dictionary
            # Key: account_number, Value: BankAccount object
            self.accounts[new_account.account_number] = new_account
            
            # AUTO-SAVE after creating account
            self.save_accounts()
            
            # DISPLAY confirmation with account details
            print(f"\n✓ Account created successfully!")
            print(f"Account Number: {new_account.account_number}")
            print(f"Account Holder: {new_account.holder_name}")
            print(f"Account Type: {new_account.account_type.capitalize()}")
            print(f"Initial Balance: ${new_account.balance:.2f}")
            
            return new_account
            
        except Exception as e:
            print(f"Error creating account: {e}")
            return None
    
    def get_account(self, account_number):
        """
        Retrieve an account by account number.
        
        Args:
            account_number (int): The account number to search for
            
        Returns:
            BankAccount: The account object or None if not found
        
        Uses dictionary.get() for safe access - returns None
        if key doesn't exist instead of raising KeyError.
        """
        # USE .get() method for safe dictionary access
        account = self.accounts.get(account_number)
        
        if account is None:
            print(f"Error: Account {account_number} not found!")
        
        return account
    
    def display_all_accounts(self):
        """
        Display summary of all accounts in a formatted table.
        
        Provides a quick overview of:
        - Account number
        - Holder name
        - Account type
        - Current balance
        """
        # CHECK if there are any accounts
        if not self.accounts:
            print("No accounts in the system.")
            return
        
        # HEADER
        print("\n" + "=" * 80)
        print(f"{'Account #':<12} {'Holder Name':<25} {'Type':<12} {'Balance':<15}")
        print("=" * 80)
        
        # ITERATE through all accounts
        for account_num, account in self.accounts.items():
            print(f"{account_num:<12} {account.holder_name:<25} "
                  f"{account.account_type.capitalize():<12} ${account.balance:<14.2f}")
        
        # FOOTER with total count
        print("=" * 80)
        print(f"Total Accounts: {len(self.accounts)}")
    
    def delete_account(self, account_number):
        """
        Delete an account from the system.
        
        Args:
            account_number (int): Account number to delete
            
        Returns:
            bool: True if successful, False otherwise
        
        Safety Feature:
        - Warns if account has balance
        - Requires confirmation before deletion
        """
        # CHECK if account exists
        if account_number in self.accounts:
            account = self.accounts[account_number]
            
            # SAFETY CHECK: Warn if account has balance
            if account.balance > 0:
                print(f"Warning: Account has balance of ${account.balance:.2f}")
                confirm = input("Are you sure you want to delete? (yes/no): ")
                
                # Require explicit 'yes' confirmation
                if confirm.lower() != 'yes':
                    print("Account deletion cancelled.")
                    return False
            
            # DELETE account from dictionary
            del self.accounts[account_number]
            
            # SAVE changes to file
            self.save_accounts()
            
            print(f"✓ Account {account_number} deleted successfully.")
            return True
        else:
            print(f"Error: Account {account_number} not found!")
            return False
    
    def generate_system_report(self):
        """
        Generate comprehensive system-wide report.
        
        This method provides insights into:
        - Total number of accounts by type
        - Total system balance
        - Average balance per account
        - Total transactions processed
        - Account with highest balance
        
        Uses Python's built-in functions like sum(), max(), etc.
        """
        # CHECK if there are accounts
        if not self.accounts:
            print("No accounts in the system.")
            return
        
        # HEADER
        print("\n" + "=" * 80)
        print("BANK SYSTEM COMPREHENSIVE REPORT")
        print("=" * 80)
        
        # INITIALIZE counters
        total_balance = 0
        all_transactions = 0
        account_types = {'savings': 0, 'current': 0}
        
        # CALCULATE statistics by iterating through all accounts
        for account in self.accounts.values():
            total_balance += account.balance
            all_transactions += len(account.transaction_history)
            account_types[account.account_type] += 1
        
        # DISPLAY account statistics
        print(f"\nTotal Accounts: {len(self.accounts)}")
        print(f"  - Savings Accounts: {account_types['savings']}")
        print(f"  - Current Accounts: {account_types['current']}")
        
        # DISPLAY financial statistics
        print(f"\nTotal System Balance: ${total_balance:.2f}")
        print(f"Average Balance per Account: ${total_balance/len(self.accounts):.2f}")
        print(f"Total Transactions Processed: {all_transactions}")
        
        # FIND account with highest balance using max() with key function
        if self.accounts:
            richest_account = max(self.accounts.values(), key=lambda acc: acc.balance)
            print(f"\nHighest Balance Account:")
            print(f"  {richest_account.holder_name} (Account #{richest_account.account_number}): "
                  f"${richest_account.balance:.2f}")
        
        # FOOTER
        print("=" * 80 + "\n")


# Confirmation message
print("✓ BankSystem class loaded successfully!")
print("  - Supports: Account management, File persistence")
print("  - Features: Save/Load with pickle, System reports")

✓ BankSystem class loaded successfully!
  - Supports: Account management, File persistence
  - Features: Save/Load with pickle, System reports


In [13]:
# ============================================================================
# HELPER FUNCTIONS FOR USER INPUT VALIDATION
# ============================================================================

def get_valid_float(prompt, min_value=0):
    """
    Get valid float input from user with validation.
    
    Args:
        prompt (str): Message to display to user
        min_value (float): Minimum acceptable value (default: 0)
        
    Returns:
        float: Valid float value entered by user
    
    This function loops until user provides valid input,
    preventing crashes from invalid data.
    """
    while True:
        try:
            # TRY to convert input to float
            value = float(input(prompt))
            
            # CHECK if value meets minimum requirement
            if value < min_value:
                print(f"Error: Value must be at least {min_value}")
                continue  # Ask again
            
            return value  # Valid input - return it
            
        except ValueError:
            # User entered non-numeric input
            print("Error: Please enter a valid number!")
            # Loop continues to ask again

def get_valid_int(prompt):
    """
    Get valid integer input from user.
    
    Args:
        prompt (str): Message to display to user
        
    Returns:
        int: Valid integer value entered by user
    
    Similar to get_valid_float but for integers (account numbers).
    """
    while True:
        try:
            value = int(input(prompt))
            return value
        except ValueError:
            print("Error: Please enter a valid number!")

# ============================================================================
# MAIN MENU FUNCTION
# ============================================================================

def main_menu():
    """
    Display interactive menu and handle user choices.
    
    This is the main entry point for the banking system.
    It provides a text-based menu interface for all operations:
    1. Account creation
    2. View account details
    3-5. Transactions (deposit, withdraw, transfer)
    6-7. Reports (history, statistics)
    8-9. System-wide operations
    10. Account deletion
    0. Exit
    
    The function uses a while loop to keep showing the menu
    until the user chooses to exit.
    """
    # CREATE or LOAD BankSystem instance
    bank = BankSystem('bank_accounts.pkl')
    
    # MAIN LOOP - continues until user exits
    while True:
        # ====================================================================
        # DISPLAY MENU
        # ====================================================================
        print("\n" + "=" * 60)
        print("       BANK ACCOUNT MANAGEMENT SYSTEM")
        print("=" * 60)
        print("1. Open a New Account")
        print("2. View Account Details")
        print("3. Deposit Money")
        print("4. Withdraw Money")
        print("5. Transfer Money")
        print("6. View Transaction History")
        print("7. View Account Statistics")
        print("8. View All Accounts")
        print("9. Generate System Report")
        print("10. Delete Account")
        print("0. Exit")
        print("=" * 60)
        
        # GET user choice
        choice = input("Enter your choice (0-10): ").strip()
        
        # ====================================================================
        # OPTION 1: Open New Account
        # ====================================================================
        if choice == '1':
            print("\n--- Open New Account ---")
            
            # GET account holder name
            name = input("Enter account holder name: ").strip()
            if not name:
                print("Error: Name cannot be empty!")
                continue  # Back to menu
            
            # GET account type
            print("Account Types: 1. Savings  2. Current")
            acc_type_choice = input("Enter account type (1 or 2): ").strip()
            
            if acc_type_choice == '1':
                acc_type = 'savings'
            elif acc_type_choice == '2':
                acc_type = 'current'
            else:
                print("Error: Invalid account type!")
                continue
            
            # GET initial balance
            initial_balance = get_valid_float("Enter initial deposit amount: $", min_value=0)
            
            # CREATE account
            bank.create_account(name, acc_type, initial_balance)
        
        # ====================================================================
        # OPTION 2: View Account Details
        # ====================================================================
        elif choice == '2':
            print("\n--- View Account Details ---")
            
            # GET account number
            acc_num = get_valid_int("Enter account number: ")
            
            # RETRIEVE account
            account = bank.get_account(acc_num)
            
            # DISPLAY details if account found
            if account:
                details = account.get_account_details()
                print("\n" + "=" * 50)
                for key, value in details.items():
                    print(f"{key}: {value}")
                print("=" * 50)
        
        # ====================================================================
        # OPTION 3: Deposit Money
        # ====================================================================
        elif choice == '3':
            print("\n--- Deposit Money ---")
            
            # GET account number
            acc_num = get_valid_int("Enter account number: ")
            
            # RETRIEVE account
            account = bank.get_account(acc_num)
            
            # PERFORM deposit if account exists
            if account:
                amount = get_valid_float("Enter deposit amount: $", min_value=0.01)
                
                # If deposit successful, save changes
                if account.deposit(amount):
                    bank.save_accounts()
        
        # ====================================================================
        # OPTION 4: Withdraw Money
        # ====================================================================
        elif choice == '4':
            print("\n--- Withdraw Money ---")
            
            # GET account number
            acc_num = get_valid_int("Enter account number: ")
            
            # RETRIEVE account
            account = bank.get_account(acc_num)
            
            # PERFORM withdrawal if account exists
            if account:
                amount = get_valid_float("Enter withdrawal amount: $", min_value=0.01)
                
                # If withdrawal successful, save changes
                if account.withdraw(amount):
                    bank.save_accounts()
        
        # ====================================================================
        # OPTION 5: Transfer Money
        # ====================================================================
        elif choice == '5':
            print("\n--- Transfer Money ---")
            
            # GET sender account
            from_acc_num = get_valid_int("Enter your account number: ")
            from_account = bank.get_account(from_acc_num)
            
            # Only proceed if sender account exists
            if from_account:
                # GET recipient account
                to_acc_num = get_valid_int("Enter recipient account number: ")
                to_account = bank.get_account(to_acc_num)
                
                # Only proceed if recipient account exists
                if to_account:
                    amount = get_valid_float("Enter transfer amount: $", min_value=0.01)
                    
                    # If transfer successful, save changes
                    if from_account.transfer(to_account, amount):
                        bank.save_accounts()
        
        # ====================================================================
        # OPTION 6: View Transaction History
        # ====================================================================
        elif choice == '6':
            print("\n--- View Transaction History ---")
            
            # GET account number
            acc_num = get_valid_int("Enter account number: ")
            
            # RETRIEVE account
            account = bank.get_account(acc_num)
            
            # DISPLAY transaction history if account exists
            if account:
                account.view_transaction_history()
        
        # ====================================================================
        # OPTION 7: View Account Statistics
        # ====================================================================
        elif choice == '7':
            print("\n--- View Account Statistics ---")
            
            # GET account number
            acc_num = get_valid_int("Enter account number: ")
            
            # RETRIEVE account
            account = bank.get_account(acc_num)
            
            # DISPLAY statistics if account exists
            if account:
                account.display_statistics()
        
        # ====================================================================
        # OPTION 8: View All Accounts
        # ====================================================================
        elif choice == '8':
            print("\n--- All Accounts ---")
            bank.display_all_accounts()
        
        # ====================================================================
        # OPTION 9: Generate System Report
        # ====================================================================
        elif choice == '9':
            print("\n--- System Report ---")
            bank.generate_system_report()
        
        # ====================================================================
        # OPTION 10: Delete Account
        # ====================================================================
        elif choice == '10':
            print("\n--- Delete Account ---")
            
            # GET account number to delete
            acc_num = get_valid_int("Enter account number to delete: ")
            
            # DELETE account (with confirmation if it has balance)
            bank.delete_account(acc_num)
        
        # ====================================================================
        # OPTION 0: Exit
        # ====================================================================
        elif choice == '0':
            # SAVE all data before exiting
            print("\n✓ Thank you for using Bank Account Management System!")
            print("✓ All data has been saved.")
            break  # Exit the while loop
        
        # ====================================================================
        # INVALID CHOICE
        # ====================================================================
        else:
            print("Error: Invalid choice! Please enter a number between 0-10.")
        
        # PAUSE before showing menu again
        # Gives user time to read output
        input("\nPress Enter to continue...")


# Confirmation message
print("✓ Menu system loaded successfully!")
print("\nTo start the banking system, run: main_menu()")

✓ Menu system loaded successfully!

To start the banking system, run: main_menu()


In [14]:
"""
============================================================================
SAMPLE INPUT/OUTPUT SCENARIOS
============================================================================

This cell demonstrates the functionality of the Bank Account Management System
with various test scenarios. Each scenario shows expected inputs and outputs.

SCENARIOS COVERED:
1. Creating new accounts
2. Performing deposits
3. Performing withdrawals (valid and invalid)
4. Transferring money between accounts
5. Viewing transaction history
6. Viewing account statistics
7. Generating system reports
8. Testing error handling
============================================================================
"""

print("=" * 80)
print("DEMONSTRATION: BANK ACCOUNT MANAGEMENT SYSTEM")
print("=" * 80)
print("\nThis demo will showcase all features of the banking system.\n")

# Create a demo bank system with a separate file
demo_bank = BankSystem('demo_accounts.pkl')

# ============================================================================
# SCENARIO 1: Creating Multiple Accounts
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 1: Creating New Accounts")
print("=" * 80)
print("\nCreating three different accounts with various initial balances...\n")

# Account 1: Savings account with initial deposit
print("INPUT: Create account for 'Alice Johnson', Type: Savings, Initial Balance: $5000")
acc1 = demo_bank.create_account("Alice Johnson", "savings", 5000)

# Account 2: Current account with initial deposit
print("\nINPUT: Create account for 'Bob Smith', Type: Current, Initial Balance: $3000")
acc2 = demo_bank.create_account("Bob Smith", "current", 3000)

# Account 3: Savings account with smaller balance
print("\nINPUT: Create account for 'Charlie Brown', Type: Savings, Initial Balance: $1500")
acc3 = demo_bank.create_account("Charlie Brown", "savings", 1500)

print("\n✓ Three accounts created successfully!")

# ============================================================================
# SCENARIO 2: Depositing Money
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 2: Depositing Money")
print("=" * 80)
print("\nMaking deposits to Alice's and Bob's accounts...\n")

if acc1 and acc2:
    # Deposit to Account 1
    print(f"INPUT: Deposit $1500 to Account {acc1.account_number} (Alice)")
    acc1.deposit(1500)
    
    # Deposit to Account 2
    print(f"\nINPUT: Deposit $2000 to Account {acc2.account_number} (Bob)")
    acc2.deposit(2000)
    
    demo_bank.save_accounts()
    print("\n✓ Deposits completed and saved!")

# ============================================================================
# SCENARIO 3: Withdrawing Money (Valid)
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 3: Withdrawing Money (Valid Transaction)")
print("=" * 80)
print("\nAlice withdraws $500 from her account...\n")

if acc1:
    print(f"Current balance: ${acc1.balance:.2f}")
    print(f"INPUT: Withdraw $500 from Account {acc1.account_number}")
    acc1.withdraw(500)
    demo_bank.save_accounts()

# ============================================================================
# SCENARIO 4: Withdrawing Money (Invalid - Insufficient Funds)
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 4: Withdrawing Money (Invalid - Insufficient Funds)")
print("=" * 80)
print("\nCharlie attempts to withdraw more than his balance...\n")

if acc3:
    print(f"Current balance: ${acc3.balance:.2f}")
    print(f"INPUT: Withdraw $5000 from Account {acc3.account_number}")
    print("EXPECTED OUTPUT: Error message due to insufficient funds\n")
    acc3.withdraw(5000)  # This will fail

# ============================================================================
# SCENARIO 5: Transferring Money Between Accounts
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 5: Transferring Money Between Accounts")
print("=" * 80)
print("\nAlice transfers $1000 to Bob's account...\n")

if acc1 and acc2:
    print(f"Alice's balance before: ${acc1.balance:.2f}")
    print(f"Bob's balance before: ${acc2.balance:.2f}")
    print(f"\nINPUT: Transfer $1000 from Account {acc1.account_number} to Account {acc2.account_number}")
    acc1.transfer(acc2, 1000)
    
    print(f"\nAlice's balance after: ${acc1.balance:.2f}")
    print(f"Bob's balance after: ${acc2.balance:.2f}")
    
    demo_bank.save_accounts()

# ============================================================================
# SCENARIO 6: Invalid Transfer (Same Account)
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 6: Invalid Transfer (Same Account)")
print("=" * 80)
print("\nAttempting to transfer to the same account...\n")

if acc1:
    print(f"INPUT: Transfer $500 from Account {acc1.account_number} to Account {acc1.account_number}")
    print("EXPECTED OUTPUT: Error message\n")
    acc1.transfer(acc1, 500)  # This will fail

# ============================================================================
# SCENARIO 7: Adding More Transactions for Better Statistics
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 7: Multiple Transactions for Statistics")
print("=" * 80)
print("\nPerforming various transactions on Bob's account...\n")

if acc2:
    print("Performing: Deposit $500, Withdraw $200, Deposit $750, Withdraw $100")
    acc2.deposit(500)
    acc2.withdraw(200)
    acc2.deposit(750)
    acc2.withdraw(100)
    demo_bank.save_accounts()

# ============================================================================
# SCENARIO 8: Viewing Transaction History
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 8: Viewing Transaction History")
print("=" * 80)
print("\nDisplaying complete transaction history for Alice's account...\n")

if acc1:
    print(f"INPUT: View transaction history for Account {acc1.account_number}")
    acc1.view_transaction_history()

# ============================================================================
# SCENARIO 9: Viewing Account Statistics with NumPy
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 9: Account Statistics (Using NumPy)")
print("=" * 80)
print("\nGenerating statistical analysis for Bob's account...\n")

if acc2:
    print(f"INPUT: Generate statistics for Account {acc2.account_number}")
    acc2.display_statistics()

# ============================================================================
# SCENARIO 10: Viewing All Accounts
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 10: Viewing All Accounts")
print("=" * 80)
print("\nDisplaying summary of all accounts in the system...\n")

print("INPUT: Display all accounts")
demo_bank.display_all_accounts()

# ============================================================================
# SCENARIO 11: System-Wide Report
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 11: System-Wide Report")
print("=" * 80)
print("\nGenerating comprehensive system report...\n")

print("INPUT: Generate system report")
demo_bank.generate_system_report()

# ============================================================================
# SCENARIO 12: Viewing Account Details
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 12: Viewing Detailed Account Information")
print("=" * 80)

if acc1:
    print(f"\nINPUT: View details for Account {acc1.account_number}")
    print("OUTPUT:")
    details = acc1.get_account_details()
    print("\n" + "=" * 50)
    for key, value in details.items():
        print(f"{key}: {value}")
    print("=" * 50)

# ============================================================================
# SCENARIO 13: Error Handling - Invalid Deposit
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 13: Error Handling - Negative Deposit")
print("=" * 80)
print("\nAttempting to deposit a negative amount...\n")

if acc1:
    print(f"INPUT: Deposit -$100 to Account {acc1.account_number}")
    print("EXPECTED OUTPUT: Error message\n")
    acc1.deposit(-100)  # This will fail

# ============================================================================
# SCENARIO 14: Error Handling - Invalid Account Creation
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 14: Error Handling - Invalid Account Creation")
print("=" * 80)
print("\nAttempting to create account with invalid data...\n")

print("INPUT: Create account with empty name")
print("EXPECTED OUTPUT: Error message\n")
demo_bank.create_account("", "savings", 1000)  # This will fail

print("\nINPUT: Create account with invalid type 'checking'")
print("EXPECTED OUTPUT: Error message\n")
demo_bank.create_account("Test User", "checking", 1000)  # This will fail

print("\nINPUT: Create account with negative balance")
print("EXPECTED OUTPUT: Error message\n")
demo_bank.create_account("Test User", "savings", -500)  # This will fail

# ============================================================================
# SCENARIO 15: File Persistence Test
# ============================================================================
print("\n" + "=" * 80)
print("SCENARIO 15: File Persistence Test")
print("=" * 80)
print("\nTesting if data persists across program sessions...\n")

print("Step 1: Current system has", len(demo_bank.accounts), "accounts")
print("Step 2: Creating new BankSystem instance (simulating program restart)")

# Create new instance - should load from file
demo_bank2 = BankSystem('demo_accounts.pkl')

print("Step 3: Loaded", len(demo_bank2.accounts), "accounts from file")
print("\n✓ File persistence working correctly!")

# Display loaded accounts to confirm
demo_bank2.display_all_accounts()

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("DEMONSTRATION COMPLETE")
print("=" * 80)
print("\nAll scenarios completed successfully! The system demonstrates:")
print("  ✓ Account creation and management")
print("  ✓ Transaction processing (deposit, withdraw, transfer)")
print("  ✓ Error handling and validation")
print("  ✓ Transaction history tracking")
print("  ✓ Statistical analysis using NumPy")
print("  ✓ File persistence using pickle")
print("  ✓ System-wide reporting")
print("\nYou can now run main_menu() for interactive use!")
print("=" * 80)

DEMONSTRATION: BANK ACCOUNT MANAGEMENT SYSTEM

This demo will showcase all features of the banking system.

No existing account file found. Starting fresh.

SCENARIO 1: Creating New Accounts

Creating three different accounts with various initial balances...

INPUT: Create account for 'Alice Johnson', Type: Savings, Initial Balance: $5000
✓ Saved 1 accounts to file.

✓ Account created successfully!
Account Number: 1000
Account Holder: Alice Johnson
Account Type: Savings
Initial Balance: $5000.00

INPUT: Create account for 'Bob Smith', Type: Current, Initial Balance: $3000
✓ Saved 2 accounts to file.

✓ Account created successfully!
Account Number: 1001
Account Holder: Bob Smith
Account Type: Current
Initial Balance: $3000.00

INPUT: Create account for 'Charlie Brown', Type: Savings, Initial Balance: $1500
✓ Saved 3 accounts to file.

✓ Account created successfully!
Account Number: 1002
Account Holder: Charlie Brown
Account Type: Savings
Initial Balance: $1500.00

✓ Three accounts creat

In [ ]:
"""
============================================================================
INTERACTIVE BANKING SYSTEM
============================================================================

Run this cell to start the interactive menu-driven banking system.
You can perform all operations through the menu interface.

AVAILABLE OPERATIONS:
- Create new accounts
- View account details
- Deposit/Withdraw/Transfer money
- View transaction history
- View statistics
- Generate reports
- Delete accounts
- Exit and save

Press CTRL+C to force exit if needed.
============================================================================
"""

# Start the interactive banking system
main_menu()

No existing account file found. Starting fresh.

       BANK ACCOUNT MANAGEMENT SYSTEM
1. Open a New Account
2. View Account Details
3. Deposit Money
4. Withdraw Money
5. Transfer Money
6. View Transaction History
7. View Account Statistics
8. View All Accounts
9. Generate System Report
10. Delete Account
0. Exit


Enter your choice (0-10):  1



--- Open New Account ---


Enter account holder name:  Gagan


Account Types: 1. Savings  2. Current


Enter account type (1 or 2):  1
Enter initial deposit amount: $ 10


✓ Saved 1 accounts to file.

✓ Account created successfully!
Account Number: 1003
Account Holder: Gagan
Account Type: Savings
Initial Balance: $10.00



Press Enter to continue... 



       BANK ACCOUNT MANAGEMENT SYSTEM
1. Open a New Account
2. View Account Details
3. Deposit Money
4. Withdraw Money
5. Transfer Money
6. View Transaction History
7. View Account Statistics
8. View All Accounts
9. Generate System Report
10. Delete Account
0. Exit


In [ ]:
"""
============================================================================
BANK ACCOUNT MANAGEMENT SYSTEM - DOCUMENTATION
============================================================================

PROJECT OVERVIEW:
-----------------
This is a complete banking system built in Python that demonstrates:
- Object-Oriented Programming (OOP)
- File handling with pickle
- Data structures (dictionaries, lists)
- NumPy for statistical calculations
- Error handling and input validation
- Interactive menu systems

KEY FEATURES:
-------------
1. ACCOUNT MANAGEMENT
   - Create savings or current accounts
   - Auto-generated unique account numbers
   - Store account holder information

2. TRANSACTIONS
   - Deposit: Add money to accounts
   - Withdraw: Remove money (with balance checking)
   - Transfer: Move money between accounts
   - All transactions are recorded with timestamps

3. FILE PERSISTENCE
   - Data saved to 'bank_accounts.pkl' file
   - Automatic loading on program start
   - Maintains data between sessions

4. REPORTING & ANALYTICS
   - Complete transaction history
   - Statistical analysis using NumPy:
     * Average transaction amount
     * Total deposits/withdrawals
     * Standard deviation
     * Min/max transactions
   - System-wide reports

5. ERROR HANDLING
   - Input validation for all operations
   - Prevents invalid transactions
   - User-friendly error messages

CLASSES:
--------
1. BankAccount
   - Represents individual bank accounts
   - Methods: deposit(), withdraw(), transfer()
   - Tracks transaction history
   - Generates statistics

2. BankSystem
   - Manages multiple accounts
   - Handles file operations (save/load)
   - Creates and deletes accounts
   - Generates system reports

USAGE EXAMPLES:
---------------

# Example 1: Creating accounts programmatically
bank = BankSystem('my_bank.pkl')
account = bank.create_account("John Doe", "savings", 1000)

# Example 2: Performing transactions
account.deposit(500)
account.withdraw(200)

# Example 3: Transferring between accounts
account1.transfer(account2, 300)

# Example 4: Viewing statistics
account.display_statistics()

# Example 5: Interactive mode
main_menu()

FILE STRUCTURE:
---------------
bank_accounts.pkl - Main data file (pickle format)
  Contains:
  - Dictionary of all accounts
  - Account counter for unique IDs

TECHNOLOGIES USED:
------------------
- Python 3.x
- pickle: Object serialization
- NumPy: Statistical calculations
- datetime: Timestamps
- os: File operations

ERROR CODES & MESSAGES:
-----------------------
"Insufficient funds" - Withdrawal/transfer exceeds balance
"Invalid recipient account" - Transfer to non-existent account
"Amount must be positive" - Negative deposit/withdrawal attempt
"Account not found" - Invalid account number provided
"Name cannot be empty" - Empty name in account creation

NUMPY FUNCTIONS USED:
----------------------
- np.sum(): Calculate totals
- np.mean(): Calculate averages
- np.max(): Find maximum values
- np.min(): Find minimum values
- np.std(): Calculate standard deviation

BEST PRACTICES DEMONSTRATED:
----------------------------
1. Input validation on all user inputs
2. Exception handling with try-except blocks
3. Clear and descriptive variable names
4. Comprehensive docstrings for all functions
5. Separation of concerns (classes for different responsibilities)
6. Data persistence between sessions
7. User-friendly error messages
8. Type hints in function signatures (could be added)

FUTURE ENHANCEMENTS:
--------------------
- User authentication with passwords
- Interest calculation for savings accounts
- Transaction limits per day
- Account statements (PDF export)
- GUI interface
- Database integration (SQLite/PostgreSQL)
- Multiple currency support
- Scheduled transactions

TESTING:
--------
All features have been tested with:
- Valid inputs
- Invalid inputs (negative amounts, insufficient funds)
- Edge cases (same account transfer, empty names)
- File persistence (save and reload)

============================================================================
"""

print("=" * 80)
print("DOCUMENTATION LOADED")
print("=" * 80)
print("\nTo view this documentation, check the cell output above.")
print("To start using the system, run: main_menu()")
print("=" * 80)